# Sistemas Inteligentes 2021/2022

## Mini-projeto 1: Pacman comilão

<img src="pacman.png" alt="Drawing" style="width: 100px;"/>

## Grupo: 51

### Elementos do Grupo

Número: 56926    Nome: Lucas Pinto   
Número: 56895    Nome: Matilde Silva    
Número: 56941    Nome: Bruno Gonzalez

(Nota: Neste relatório pode adicionar as células de texto e código que achar necessárias.)

(descreva aqui, textualmente, como decidiu representar os estados em Python; ilustre nas células de código abaixo a representação em Python de um estado à sua escolha)

## Representação dos estados

Para representar os estados decidimos criar a classe `PacmanEstado`.

O construtor desta classe recebe como parâmetros:
- `pacman` —  Tuplo (x,y) com a posição do Pacman.
- `gums` — Dicionário contendo chaves associadas a posições do mapa e respetivos valores representando o tipo da pastilha.
- `cellsVisited` — Dicionário em que as chaves estão associadas a posições do mapa que já foram visitadas, e os valores correspondem à quantidade de vezes que estas já foram visitadas.
- `points` — Int com a quantidade de pontos obtidos no determinado estado.

Métodos:
- `__init__(pacman=(1, 1), gums=None, cellsVisited=None, points=0)` — Construtor de classe que recebe os objetos referidos acima.
- `__lt__(other: PacmanEstado) -> bool` — Retorna um booleano que compara se a quantidade de pontos de um PacmanEstado é superior a outro.
- `__eq__(other: PacmanEstado) -> bool` — Retorna um booleano que compara se o objeto é igual a outro.
- `__hash__() -> int` — Retorna um int único que representa o estado.
- `visitCell(cell: Tuple[int, int])` — Visita uma célula e come a pastilha dessa célula, se existir.
- `eatGum(gum: Tuple[int, int])` — Come a pastilha, alterando os pontos do estado.

**Representação de um estado à sua escolha**
```
pacman: (2, 1)
pastilhas: {(5, 8): 'N', (7, 3): 'C', (4, 5): 'D'}
points: 1
cellsVisited: {(1, 1): 1, (2, 1): 1}
```

**Desenhado, ficaria da seguinte forma:**
```
= = = = = = = = = = 
= . @ . . . . . . = 
= . = = = = = = . = 
= . = . . . = C . = 
= . = . . . = . . = 
= . = . D . = . . = 
= . = . . . . . . = 
= . . . . . . . . = 
= . . . . N . . . = 
= = = = = = = = = = 
```




In [1]:
# se definiu uma classe para representar os estados, inclua aqui o código Python correspondente

from typing import Tuple
from __future__ import annotations # Para poder usar o tipo PacmanEstado nos métodos antes da classe estar definida

class PacmanEstado:
    def __init__(self, pacman=(1, 1), gums=None, cellsVisited=None, points=0, moves=0):
        self.pacman = pacman
        self.points = points
        self.moves = moves

        if gums is None:
            gums = {}

        self.gums = gums

        if cellsVisited is None:
            self.cellsVisited = {pacman: 1}
        else:
            self.cellsVisited = cellsVisited
        """ cellsVisited
            Key: (x,y) of visited cell
            Value: How many times it was visited
        """

    def __lt__(self, other: PacmanEstado) -> bool:
        """Compara a quantidade de pontos de self e other"""
        return isinstance(other, PacmanEstado) and self.points < other.points

    def __eq__(self, other: PacmanEstado) -> bool:
        """Compara se self é igual a other"""
        return isinstance(other, PacmanEstado) \
               and self.pacman == other.pacman \
               and self.points == other.points \
               and tuple(self.gums) == tuple(other.gums) \
               and tuple(self.cellsVisited.items()) == tuple(other.cellsVisited.items())

    def __hash__(self) -> int:
        """"Hash do estado"""
        return hash((self.pacman, self.points, tuple(self.gums), tuple(self.cellsVisited.items())))

    def copy(self) -> PacmanEstado:
        """Efetua uma cópia do estado"""
        return PacmanEstado(self.pacman, self.gums.copy(), self.cellsVisited.copy(), self.points, self.moves)

    def visitCell(self, cell: Tuple[int, int]):
        """Visita a célula e come a pastilha, se existir"""
        self.pacman = cell
        self.moves += 1

        if cell not in self.cellsVisited:
            self.cellsVisited[cell] = 0

        self.cellsVisited[cell] += 1

        if cell in self.gums:
            self.eatGum(cell)

    def eatGum(self, gum: Tuple[int, int]):
        """"Come a pastilha e altera a pontuação"""
        if self.gums[gum] == 'N':
            self.points += 1
        elif self.gums[gum] == 'D':
            self.points += max(0, 5 - self.moves)
        elif self.gums[gum] == 'C':
            self.points += self.moves

        del self.gums[gum]

## Formulação do problema

Um problema deste tipo é uma instância de `PacmanPastilhas`.

O seu construtor recebe os parâmetros:
- `pacman` — Tuplo (x,y) com a posição do Pacman.
- `goal` — Int do pontos-objetivo.
- `gums` — Dicionário contendo chaves associadas a posições do mapa e respetivos valores representando o tipo da pastilha.
- `obstacles` — Dicionário em que as chaves representam as posições que o Pacman não pode utilizar.
- `dim` — Int da dimensão do mapa.

para além destes, também são inicializados os parâmetros:
- `timeStart` — Inicialização do contador de tempo utilizado nas pastilhas que requerem este valor TODO mudar para qtd moves
- `directions` — Dicionário em que as chaves são as quatro direcções ortogonais e os valores o incremento dx e dy a aplicar.

Esta classe possui funções:
- `actions(state: PacmanEstado)` — Retorna as possíveis ações a partir do estado.
- `result(state: PacmanEstado, action: str)` — Devolve uma nova instância do estado com a ação aplicada.
- `goal_test(state: PacmanEstado)` — Determina se o estado é solução.
- `path_cost(c: int, state1: PacmanEstado, action: str, state2: PacmanEstado)` — Retorna a soma do custo acumulado com o custo de aplicar a ação `action` ao estado `state1`.
- `exec(state: PacmanEstado, actions: List[str])` — Recorrendo a funções anteriores (`result`, `path_cost`, e `goal_test`) para calcular todo o custo que foi gerado no desenrolar do algoritmo até ao ponto que se pretende saber.
- `display(state: PacmanEstado)` — Representação textual do mapa do estado usando.

In [2]:
from searchPlus import *
from typing import Tuple, List

class PacmanPastilhas(Problem):
    def __init__(self, pacman=(1, 1), goal=1, gums={}, obstacles={}, dim=10):
        super().__init__(PacmanEstado(pacman, gums), goal)
        self.dim = dim
        self.obstacles = obstacles
        self.directions = {"N": (0, -1), "W": (-1, 0), "E": (1, 0), "S": (0, 1)}

    def actions(self, state: PacmanEstado) -> List[str]:
        """
        Retorna as ações que podem ser executadas num dado estado.
        """
        def valid(x: int, y: int) -> bool:
            return (x, y) not in self.obstacles\
                    and x >= 0 and y >= 0\
                    and x <= self.dim and y <= self.dim

        x, y = state.pacman
        return [act for act in self.directions
                if valid(x + self.directions[act][0], y + self.directions[act][1])]

    def result(self, state: PacmanEstado, action: str) -> PacmanEstado:
        """
        Retorna o estado que resulta de executar uma dada ação num
        dado estado.

        pre: action in self.actions(state)
        """
        x, y = state.pacman
        dx, dy = self.directions[action]

        new_state = state.copy()
        new_state.visitCell((x+dx, y+dy))

        return new_state

    def goal_test(self, state: PacmanEstado) -> bool:
        """
        Retorna se o estado é o objetivo.
        """
        return state.points >= self.goal

    def path_cost(self, c: int, state1: PacmanEstado, action: str, state2: PacmanEstado) -> int:
        """
        Retorna o custo de uma solução que chega ao state2 através
        do state1, assumindo custo acumulado c de state1.

        pre: state2 == self.result(state1, action)
        """
        return c + state2.cellsVisited[state2.pacman]

    def exec(self, state: PacmanEstado, actions: List[str]) -> Tuple[PacmanEstado, int]:
        """
        Tuplo com o estado atual do Pacman e o respetivo custo acumulado até esta posição
        """
        custo = 0
        for a in actions:
            seg = self.result(state, a)
            custo = self.path_cost(custo, state, a, seg)
            state = seg
        self.display(state)
        print('Custo:', custo)
        print('Goal?', self.goal_test(state))
        return (state, custo)

    def display(self, state: PacmanEstado):
        """
        Constrói o mapa 2D da representação de um estado
        """
        grid = ''
        for y in range(self.dim + 1):
            for x in range(self.dim + 1):
                if (x, y) in self.obstacles:
                    grid += '= '
                elif (x, y) == state.pacman:
                    grid += '@ '
                elif (x,y) in state.gums:
                    grid += f'{state.gums[(x, y)]} '
                else:
                    grid += '. '
            grid += '\n'

        print(grid, end='')

    def display_trace(self, actions: List[str]):
        """
        Constrói o mapa 2D da representação de um estado mostrando o caminho percorrido
        """
        path = set()
        st = self.initial
        for a in actions[:-1]:
            st = self.result(st,a)
            path.add(st.pacman)

        """ print the state please"""
        output=""
        for y in range(self.dim + 1):
            for x in range(self.dim + 1):
                if (x, y) in self.obstacles:
                    output += '= '
                elif (x, y) == self.initial.pacman:
                    output += '@ '
                elif (x, y) in path:
                    output += '+ '
                elif (x,y) in self.initial.gums:
                    output += f'{self.initial.gums[(x, y)]} '
                else:
                    output += '. '
            output += "\n"
        print(output, end='')

## Criação de estados e do problema

(Mostrem que o código está a funcionar, construindo instâncias da classe **PacmanPastilhas**, fazendo display dos estados, verificando o teste do estado final, gerando as ações para alguns estados, executando ações a partir de alguns estados e gerando novos estados e mostrando a evolução dos custos; verificando que os estados não se modificam com as ações (são gerados novos estados) e que a igualdade e a comparação entre estados funciona. Mostrem que a execução de sequências de ações está a funcionar bem.)

In [3]:
# código de teste do problema
from random import randint

def line(x, y, dx, dy, length):
    """Uma linha de células de comprimento 'length' começando em (x, y) na direcção (dx, dy)."""
    return {(x + i * dx, y + i * dy) for i in range(length)}

def quadro(x, y, length):
    """Uma moldura quadrada de células de comprimento 'length' começando no topo esquerdo (x, y)."""
    length += 1 # fix para as coordenadas serem de (0, 0) a (dim, dim)
    return line(x, y, 0, 1, length)\
        | line(x + length - 1, y, 0, 1, length)\
        | line(x, y, 1, 0, length)\
        | line(x, y + length - 1, 1, 0, length)

def mostra(custo, final, acoes, acao):
    """Mostra e formata a mensagem."""
    print(f'Custo: {custo}')
    print(f'É final? {"Sim" if final else "Não"}')
    print(f'Ações: {acoes}. Escolhi {acao}')

l = line(2, 2, 1, 0, 6)
c = line(2, 3, 0, 1, 4)
d = line(6, 3, 0, 1, 3)
fronteira = quadro(0, 0, 10)

prob = PacmanPastilhas(
    pacman=(1, 1),
    goal=1,
    gums={(2,1): 'N', (5, 8): 'N', (7, 3): 'C', (4,5): 'D'},
    obstacles=fronteira | l | c | d,
    dim=10)

est = prob.initial
prob.display(est)
final = prob.goal_test(est)
acoes = prob.actions(est)
acao = acoes[randint(0, len(acoes)-1)]
custo = 0
mostra(custo, final, acoes, acao)
print('------------------------')

# Mostrar estados até chegar ao objetivo ou 500 iterações
for i in range(500):
    novo_est = prob.result(est, acao)
    prob.display(novo_est)
    final = prob.goal_test(novo_est)
    acoes = prob.actions(novo_est)
    acao = acoes[randint(0, len(acoes)-1)]
    # Não precisamos do estado anterior nem da acao
    custo = prob.path_cost(custo, est, None, novo_est)
    mostra(custo, final, acoes, acao)
    print(f'São o mesmo estado? {"Sim" if est == novo_est else "Não"}')
    print(f'O estado anterior é menor? {"Sim" if est < novo_est else "Não"}')
    print('------------------------')

    if final:
        break
    est = novo_est


= = = = = = = = = = = 
= @ N . . . . . . . = 
= . = = = = = = . . = 
= . = . . . = C . . = 
= . = . . . = . . . = 
= . = . D . = . . . = 
= . = . . . . . . . = 
= . . . . . . . . . = 
= . . . . N . . . . = 
= . . . . . . . . . = 
= = = = = = = = = = = 
Custo: 0
É final? Não
Ações: ['E', 'S']. Escolhi E
------------------------
= = = = = = = = = = = 
= . @ . . . . . . . = 
= . = = = = = = . . = 
= . = . . . = C . . = 
= . = . . . = . . . = 
= . = . D . = . . . = 
= . = . . . . . . . = 
= . . . . . . . . . = 
= . . . . N . . . . = 
= . . . . . . . . . = 
= = = = = = = = = = = 
Custo: 1
É final? Sim
Ações: ['W', 'E']. Escolhi W
São o mesmo estado? Não
O estado anterior é menor? Sim
------------------------


## Teste de procura de solução

(utilização de algoritmos de procura aprendidos nas aulas e comparação dos resultados ao nível de tempo de execução e solução obtida; comente aqui os resultados obtidos e o que observa)

In [4]:
# Algoritmos de procura
#   Custo Uniforme em Árvore
def best_first_tree_search(problem, f):
    f = memoize(f, 'f')
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node
    frontier = PriorityQueue(min, f)
    frontier.append(node)
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        for child in node.expand(problem):
            frontier.append(child)
    return None

def uniform_cost_tree_search(problem):
    return best_first_tree_search(problem, lambda node: node.path_cost)
#    -------

#    Largura em Grafo
def breadth_first_graph_search(problem):
    """Search the deepest nodes in the search tree first."""
    return graph_search(problem, FIFOQueue())
#    -------

#   Profundidade Iterativa em Grafo
def graph_limited_search(problem, frontier,lim):
    frontier.append(Node(problem.initial))
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)
        if node.depth < lim:
            frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
    return 'cutoff'

def depth_limited_graph_search(problem, depth):
    return graph_limited_search(problem, Stack(), depth)

def iterative_deepening_plus_graph_search(problem):
    for depth in range(sys.maxsize):
        result = depth_limited_graph_search(problem, depth)
        if result != 'cutoff':
            return result
#    -------
# ----------------------------------------------

In [5]:
from timeit import default_timer
from typing import Callable

# Problema -------------------------------------
prob = PacmanPastilhas(
    pacman=(1, 1),
    goal=2,
    gums={(2,1): 'N', (5, 8): 'N', (4, 3): 'C', (4,5): 'D'},
    obstacles=fronteira | l | c | d,
    dim=10)


est = prob.initial
print('Estado inicial:')
prob.display(est)
print('\n')
# ----------------------------------------------

# Função de teste
def testa(prob: PacmanPastilhas, algo: Callable):
    start = default_timer()

    res = algo(prob)
    if not res:
        print('Sem resultado')
    else:
        sol = res.solution()
        prob.display_trace(sol)
        print(f'Ações ({len(sol)}): {"-".join(sol)}')
        print(f'Custo: {res.path_cost}')
        print(f'Pontuação: {res.state.points}')

    stop = default_timer()
    print(f'Time: {stop-start}s\n')
# ----------------------------------------------

Estado inicial:
= = = = = = = = = = = 
= @ N . . . . . . . = 
= . = = = = = = . . = 
= . = . C . = . . . = 
= . = . . . = . . . = 
= . = . D . = . . . = 
= . = . . . . . . . = 
= . . . . . . . . . = 
= . . . . N . . . . = 
= . . . . . . . . . = 
= = = = = = = = = = = 




### Procura em Árvore

#### BFS - Largura Primeiro

Este é o primeiro caminho a ser encontrado (árvore de nível 13) e tem um custo de 14.
Poderia-se ter usado um caminho mais ótimo (`S-S-S-S-S-S-E-E-N-N-N-N-E`) que tem o mesmo nível de árvore, 
mas o BFS não leva em consideração os custos das ações, e como a prioridade das ações é Norte, Oeste, Este, Sul, esta 
solução foi descoberta antes da ótima.

In [6]:
testa(prob, breadth_first_tree_search)

= = = = = = = = = = = 
= @ + . . . . . . . = 
= + = = = = = = . . = 
= + = . C . = . . . = 
= + = . . . = . . . = 
= + = . D . = . . . = 
= + = . . . . . . . = 
= + + + + + . . . . = 
= . . . . N . . . . = 
= . . . . . . . . . = 
= = = = = = = = = = = 
Ações (13): E-W-S-S-S-S-S-S-E-E-E-E-S
Custo: 14
Pontuação: 2
Time: 0.24786233399936464s



#### DFS - Profundidade Primeiro
No DFS em árvore iremos entrar em ciclo infinito porque o dicionário de células visitadas é copiado e incrementado 
na nova célula, portanto, como o dicionário mudou, o estado também será diferente.

#### DFS Progressivo - Aprofundamento Progressivo
O DFS Progressivo em árvore encontra uma solução apesar de não ser garantidamente ótima.
Procura assim a solução mais próxima da raiz, obtendo a menor quantidade de ações possível.

Tem a mesma solução que o BFS em árvore porque a primeira solução encontra-se ao nível 13, e nesse último nível, 
comporta-se como uma procura em largura.

In [7]:
testa(prob, iterative_deepening_search)

= = = = = = = = = = = 
= @ + . . . . . . . = 
= + = = = = = = . . = 
= + = . C . = . . . = 
= + = . . . = . . . = 
= + = . D . = . . . = 
= + = . . . . . . . = 
= + + + + + . . . . = 
= . . . . N . . . . = 
= . . . . . . . . . = 
= = = = = = = = = = = 
Ações (13): E-W-S-S-S-S-S-S-E-E-E-E-S
Custo: 14
Pontuação: 2
Time: 0.09507879400189267s



#### Custo Uniforme
No custo uniforme em árvore há optimalidade da solução. Neste, são primeiramente expandidos os nós com menor custo, evitando assim caminhos já percorridos, como podemos observar ao compararmos com os algoritmos anteriores, em que o primeiro passo foi sempre comer a pastilha na posição (1,2), voltado em seguida para trás, é mais custoso (14) que comer a pastilha de crescimento (13).

In [8]:
testa(prob, uniform_cost_tree_search)

= = = = = = = = = = = 
= @ N . . . . . . . = 
= + = = = = = = . . = 
= + = + C . = . . . = 
= + = + . . = . . . = 
= + = + D . = . . . = 
= + = + . . . . . . = 
= + + + . . . . . . = 
= . . . . N . . . . = 
= . . . . . . . . . = 
= = = = = = = = = = = 
Ações (13): S-S-S-S-S-S-E-E-N-N-N-N-E
Custo: 13
Pontuação: 13
Time: 0.08815080599742942s



<hr/>

### Grafos
Nas procuras em grafo não se visita estados já visitados (exceto o custo uniforme quando o custo será inferior).
Contudo, um estado deste problema inclui quantas vezes uma célula foi visitada.

Sempre que o pacman muda de posição é criado um novo estado, e quantidade de vezes que a célula foi visitada é incrementada

Portanto, como o dicionário é diferente, haverá uma maior quantidade de estados, ao invés de apenas considerar a posição do pacman no estado.


É importante referir também que para a pesquisa em grafo os estados visitados são guardados num set, e é necessário calcular a hash de cada.
Para esta última parte do cálculo, é necessário processar o dicionário de células visitadas (entre outras coisas), que se torna custoso em computação.


Para "simularmos" uma pesquisa em grafo neste problema, embora fosse uma representação incorreta do estado, não deveriamos considerar este dicionário.

Como referência, os tempos de execução das pesquisas com o dicionário são, omitindo os casos em que o tempo é idêntico:
```
BFS Grafo:
Custo: 14
Ações (13): E-W-S-S-S-S-S-S-E-E-E-E-S
Time: 19.181869682004617s

DFS Progressivo Grafo:
Custo: 13
Ações (13): S-S-S-S-S-S-E-E-E-N-N-N-N
Time: 0.14453338000021176s

Custo Uniforme Grafo:
Custo: 13
Ações (13): S-S-S-S-S-S-E-E-N-N-N-N-E
Time: 2.771507126002689s
```

E sem o dicionário:
```
BFS Grafo:
Custo: 14
Ações (13): E-W-S-S-S-S-S-S-E-E-E-E-S
Time: 0.0016931040008785203s

DFS Progressivo Grafo:
Custo: 14
Ações (13): E-W-S-S-S-S-S-S-E-E-E-S-E
Time: 0.003614033994381316s

Custo Uniforme Grafo:
Custo: 13
Ações (13): S-S-S-S-S-S-E-E-N-N-N-N-E
Time: 0.004510387996560894s
```

#### BFS - Largura Primeiro
O funcionamento deste algoritmo de pesquisa é idêntico ao BFS em árvore, com a única diferença de não visitar estados já visitados.

Pelas razões supramencionadas, usando a representação correta do estado, este algoritmo é mais lento que a sua vertente em árvore.


In [9]:
testa(prob, breadth_first_graph_search)

= = = = = = = = = = = 
= @ + . . . . . . . = 
= + = = = = = = . . = 
= + = . C . = . . . = 
= + = . . . = . . . = 
= + = . D . = . . . = 
= + = . . . . . . . = 
= + + + + + . . . . = 
= . . . . N . . . . = 
= . . . . . . . . . = 
= = = = = = = = = = = 
Ações (13): E-W-S-S-S-S-S-S-E-E-E-E-S
Custo: 14
Pontuação: 2
Time: 20.160041625000304s



#### DFS - Profundidade Primeiro
O funcionamento deste algoritmo de pesquisa é idêntico ao DFS em árvore, com a única diferença de não visitar estados já visitados.

Esta pesquisa resulta também num ciclo infinito.


#### DFS Progressivo - Aprofundamento Progressivo
O funcionamento deste algoritmo de pesquisa é idêntico ao Aprofundamento Progressivo em árvore, com a única diferença de não visitar estados já visitados.

Por este motivo, não seguiu o mesmo caminho.

In [10]:
testa(prob, iterative_deepening_plus_graph_search)

= = = = = = = = = = = 
= @ N . . . . . . . = 
= + = = = = = = . . = 
= + = . C . = . . . = 
= + = . + . = . . . = 
= + = . + . = . . . = 
= + = . + . . . . . = 
= + + + + . . . . . = 
= . . . . N . . . . = 
= . . . . . . . . . = 
= = = = = = = = = = = 
Ações (13): S-S-S-S-S-S-E-E-E-N-N-N-N
Custo: 13
Pontuação: 13
Time: 0.1606123439996736s



#### Custo Uniforme
O funcionamento deste algoritmo de pesquisa é idêntico ao Custo Uniforme em árvore, com a única diferença de não visitar estados já 
visitados que tenham um custo inferior.

Pelas razões supramencionadas, usando a representação correta do estado, este algoritmo é mais lento que a sua vertente em árvore.


In [11]:
testa(prob, uniform_cost_search)

= = = = = = = = = = = 
= @ N . . . . . . . = 
= + = = = = = = . . = 
= + = + C . = . . . = 
= + = + . . = . . . = 
= + = + D . = . . . = 
= + = + . . . . . . = 
= + + + . . . . . . = 
= . . . . N . . . . = 
= . . . . . . . . . = 
= = = = = = = = = = = 
Ações (13): S-S-S-S-S-S-E-E-N-N-N-N-E
Custo: 13
Pontuação: 13
Time: 3.125211244005186s

